In [1]:
import numpy as np
from scipy.linalg import lstsq

In [5]:
class LinearDiscriminantAnalysis:
    """Linear discriminant analysis
    """
    def fit(self, X, y):
        """Fit LinearDiscriminatAnalysis model according to the given
        training data
        
        Parameters:
        -----------
        X: array-like of shape (n_samples, n_features)
            Training data
            
        y: array-like of shape (n_samples,)
            Target values
        """
        self.classes_ = np.unique(y)
        n_classes = len(self.classes_)
        n_features = X.shape[1]
        self.priors_ = np.zeros(n_classes)
        self.means_ = np.zeros((n_classes, n_features))
        self.covariance_ = np.zeros((n_features, n_features))
        for i, c in enumerate(self.classes_):
            X_c = X[y == c]
            self.means_[i] = np.mean(X_c, axis=0)
            self.priors_[i] = len(X_c) / n_classes
            self.covariance_ += self.priors_[i] + np.cov(X_c.T, bias=True)
            
        self.coef_ = lstsq(self.covariance_, self.means_.T)[0].T # Sigma-1 * w = mu_k 
        self.intercept_ = -0.5 * np.diag(np.dot(self.means_, self.coef_).T) + np.log(self.priors_)
        if n_classes == 2:
            self.coef_ = np.atleast_2d(self.coef_[0] - self.coef_[1])
            self.intercept_ = np.atleast_1d(self.intercept_[0] - self.intercept_[1])
            
        return self
    
    
    def decision_function(self, X):
        scores = np.dot(X, self.coef_.T) + self.intercept_
        if scores.shape[0] == 1:
            return scores.ravel()
        else:
            return scores
            
        
    def predict(self, X):
        scores = self.decision_function(X)
        if len(scores) == 1:
            indices = 1 - (score>0).astype(int)
        else:
            indices = np.argmax(scores, axis=1)
        return self.classes_[indices]